In [1]:
import keras_tuner

In [2]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input as InceptionV3_Preprocess
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

#import tuners

from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

import time

In [3]:
categories = ['atypical', 'indeterminate', 'negative', 'typical']
train_image_arrays = np.load('train_image_arrays.npy', allow_pickle = True)
test_image_arrays = np.load('test_image_arrays.npy', allow_pickle = True)
train_labels = np.load('train_labels.npy', allow_pickle = True)
test_labels = np.load('test_labels.npy', allow_pickle = True)

In [4]:
LOG_DIR = f"{int(time.time())}"

In [5]:
def tl_Model(hp):
    model_inceptionv3 = InceptionV3(weights = 'imagenet', include_top = False, input_shape = train_image_arrays[0].shape, pooling = 'avg')
    model = Sequential([
        model_inceptionv3,
        Dense(hp.Int("input_units1", min_value = 32, max_value = 1024, step = 32), activation = 'relu', name = 'Dense_1'),
        Dropout(0.2),
        Dense(hp.Int("input_units2", min_value = 32, max_value = 512, step = 32), activation = 'relu', name = 'Dense_2'),
        Dropout(0.2),
        Dense(hp.Int("input_units3", min_value = 16, max_value = 64, step = 16), activation = 'relu', name = 'Dense_3'),
        Dropout(0.2),
        Dense(4, activation = 'softmax')
    ])
    model.layers[0].trainable = False
    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy']
    )

    return model

In [6]:
def get_preprocessed_input():
    '''
    Pre-processes the 15-SCENE data using Resnet50 preprocessing function.
    
    Returns the tupple of pre-processed training and test data.
    '''
    ### BEGIN SOLUTION
    # x_train_inception = preprocess_input(train_image_arrays)
    # x_test_inception  = preprocess_input(test_image_arrays)
    #Model X
    x_train_inception = InceptionV3_Preprocess(train_image_arrays)
    x_test_inception = InceptionV3_Preprocess(test_image_arrays)
    
    
    return (x_train_inception, x_test_inception)

In [7]:
train_image_inception, test_image_inception = get_preprocessed_input()

In [8]:
tuner = RandomSearch(tl_Model, objective= "val_accuracy", max_trials= 3, executions_per_trial = 3, directory = LOG_DIR)
tuner.search(x = train_image_inception, y = train_labels, epochs = 50, batch_size = 32, validation_data = (test_image_inception, test_labels))




Trial 3 Complete [00h 06m 29s]
val_accuracy: 0.550561785697937

Best val_accuracy So Far: 0.5730336904525757
Total elapsed time: 00h 19m 32s
INFO:tensorflow:Oracle triggered exit


In [12]:
print(tuner.get_best_hyperparameters()[0].values)



{'input_units1': 672, 'input_units2': 320, 'input_units3': 48}


In [10]:
# reduce_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,mode='auto', min_delta=0.00001, cooldown=0, min_lr=0,)
# checkpoint_best_only = ModelCheckpoint(filepath = 'checkpoints_best_only/checkpoint', save_best_only=True, monitor='val_accuracy', save_weights_only=True,  verbose=1)
# early_stopping = EarlyStopping(monitor='val_accuracy', patience = 3, mode = 'max', verbose = 1)
# new_history = tuner_Model.fit(train_image_inception, train_labels, epochs=50, validation_data=(test_image_inception, test_labels), callbacks=[early_stopping, checkpoint_best_only, reduce_loss])